In [35]:
from importlib import reload

import numpy as np

from ics.cobraControl import ethernet
from ics.cobraControl import func, cmds
from ics.cobraControl import convert


class PFI(object):
    CW = 1
    CCW = -1
    DISABLE = 0
    
    nCobrasPerModule = 57
    nModules = 42
    
    def __init__(self, fpgaHost='localhost', doConnect=True, doLoadModel=True):
        self.fpgaHost = fpgaHost
        if doConnect:
            self.connect()
        if doLoadModel:
            self.loadModel()
        
    def connect(self):
        ethernet.sock.connect(self.fpgaHost, 4001)
        
    def disconnect(self):
        ethernet.sock.close()
        ethernet.sock = ethernet.Sock()
        
    def loadModel(self, filename=None):
        import ics.cobraOps.CobrasCalibrationProduct as cobraModel
        reload(cobraModel)

        if filename is None:
            filename = "/Users/cloomis/Sumire/PFS/git/ics_cobraOps/python/ics/cobraOps/usedXMLFile.xml"
        
        self.calibModel = cobraModel.CobrasCalibrationProduct(filename)
        
    def _freqToPeriod(self, freq):
        """ Convert frequency to 60ns ticks """        
        return int(round(16e3 / (freq)))

    def _periodToFreq(self, freq):
        """ Convert 60ns ticks to a frequency """
        return  (16e3 / per) if (per>=1) else 0

    def _mapCobraIndex(self, cobra):
        """ Convert our module + cobra to global cobra index for the calibration product. """
        
        return 1 + ((cobra.b - 1)*self.nCobrasPerModule + cobra.c-1)
        
    def setPer(self, cobras):
        for c in cobras:
            cobraIdx = self._mapCobraIndex(c)
            thetaPer = self._freqToPeriod(self.calibModel.motorFreq1[cobraIdx]/1000)
            phiPer = self._freqToPeriod(self.calibModel.motorFreq2[cobraIdx]/1000)

            print(f'set {c.board},{c.cobra} to {thetaPer},{phiPer} {self.calibModel.motorPer1[c.cobra]}')
            c.p = func.SetParams(p0=thetaPer, p1=phiPer, en=(True, True))
        err = func.SET(cobras)
        
    def moveSteps(self, cobras, steps, dirs):
        
        if len(cobras) != len(steps):
            raise RuntimeError("number of steps must match number of cobras")
        if len(cobras) != len(dirs):
            raise RuntimeError("number of directions must match number of cobras")

        model = self.calibModel
        
        for c_i, c in enumerate(cobras):
            steps1 = steps[c_i]
            dirs1 = dirs[c_i]
            en = (steps1[0] != 0, steps1[1] != 0)
            cobraId = self._mapCobraIndex(c)
            
            if dirs1[0] == 'cw':
                ontime1 = model.motorOntimeFwd1[cobraId]
                offtime1 = model.motorOfftimeFwd1[cobraId]
            elif dirs1[0] == 'ccw':
                ontime1 = model.motorOntimeRev1[cobraId]
                offtime1 = model.motorOfftimeRev1[cobraId]
            else:
                raise ValueError(f'invalid direction: {dirs1[0]}')
                
            if dirs1[1] == 'cw':
                ontime2 = model.motorOntimeFwd2[cobraId]
                offtime2 = model.motorOfftimeFwd2[cobraId]
            elif dirs1[1] == 'ccw':
                ontime2 = model.motorOntimeRev2[cobraId]
                offtime2 = model.motorOfftimeRev2[cobraId]
            else:
                raise ValueError(f'invalid direction: {dirs1[1]}')
                
            c.p = func.RunParams(pu=(ontime1, ontime2),
                                 st=(steps1),
                                 sl=(offtime1, offtime2),
                                 en=en,
                                 dir=dirs)
        err = func.RUN(cobras)
        

def allocateAllCobras():
    return allocateCobraRange(range(1,43))

def allocateCobraRange(modules, cobraNums=None):
    """ Utility to allocate swaths of cobras:
    
    Args:
      modules (int array-like): a list of 1-indexed boards to allocate from.
      cobras  (int array-like): a list of 1-indexed cobras to allocate from.

    Return:
      cobras
    """
    cobras = []
    
    if np.isscalar(modules):
        modules = [modules]
    for m in modules:
        if m == 0:
            raise IndexError('module numbers are 1-indexed, grrr.')
        if cobraNums is None:
            _cobraNums = range(1,58)
        else:
            _cobraNums = cobraNums
            
        for c in _cobraNums:
            if c == 0:
                raise IndexError('cobra numbers are 1-indexed, grrr.')

            cobras.append(func.Cobra(b, c))
        
    return cobras

def allocateCobraList(cobraIds):
    cobras = []
    for mc in cobraIds:
        m, c = mc
        if m == 0:
            raise IndexError('module numbers are 1-indexed, grrr.')
        if c == 0:
            raise IndexError('cobra numbers are 1-indexed, grrr.')

        cobras.append(func.Cobra(b, c))
        
    return cobras

def testRuns(cobras):
    for c in cobras:
        c.p = func.RunParams(pu=(70,70),
                             st=(10,10),
                             en=(True, True),
                             dir=('cw', 'cw'))

In [37]:
testCobras = "oneModule"

if testCobras == "allCobras":
    cobras = allocateCobraRange(range(1,43))
elif testCobras == "oneModule":
    cobras = allocateCobraRange(range(1,3))
else:
    cobras = allocateCobraList([(1,1)])

NameError: name 'b' is not defined

In [30]:
pfi = PFI()

In [31]:
%pdb on
pfi.setFreq(cobras)
%pdb off

Automatic pdb calling has been turned ON
set 1,1 to 253,153 63241.0
set 1,2 to 251,154 63745.0
set 1,3 to 252,153 63492.0
set 1,4 to 253,152 63241.0
set 1,5 to 252,151 63492.0
set 1,6 to 256,151 62500.0
set 1,7 to 252,152 63492.0
set 1,8 to 250,153 64000.0
set 1,9 to 252,151 63492.0
set 1,10 to 254,149 62992.0
set 1,11 to 253,152 63241.0
set 1,12 to 254,151 62992.0
set 1,13 to 252,152 63492.0
set 1,14 to 250,151 64000.0
set 1,15 to 252,154 63492.0
set 1,16 to 252,152 63492.0
set 1,17 to 254,150 62992.0
set 1,18 to 253,152 63241.0
set 1,19 to 250,151 64000.0
set 1,20 to 251,151 63745.0
set 1,21 to 254,153 62992.0
set 1,22 to 252,153 63492.0
set 1,23 to 250,152 64000.0
set 1,24 to 253,151 63241.0
set 1,25 to 256,150 62500.0
set 1,26 to 253,153 63241.0
set 1,27 to 253,148 63241.0
set 1,28 to 252,151 63492.0
set 2,1 to 253,153 63241.0
set 2,2 to 251,154 63745.0
set 2,3 to 252,153 63492.0
set 2,4 to 253,152 63241.0
set 2,5 to 252,151 63492.0
set 2,6 to 256,151 62500.0
set 2,7 to 252,152 634

set 55,13 to 252,152 63492.0
set 55,14 to 250,151 64000.0
set 55,15 to 252,154 63492.0
set 55,16 to 252,152 63492.0
set 55,17 to 254,150 62992.0
set 55,18 to 253,152 63241.0
set 55,19 to 250,151 64000.0
set 55,20 to 251,151 63745.0
set 55,21 to 254,153 62992.0
set 55,22 to 252,153 63492.0
set 55,23 to 250,152 64000.0
set 55,24 to 253,151 63241.0
set 55,25 to 256,150 62500.0
set 55,26 to 253,153 63241.0
set 55,27 to 253,148 63241.0
set 55,28 to 252,151 63492.0
set 56,1 to 253,153 63241.0
set 56,2 to 251,154 63745.0
set 56,3 to 252,153 63492.0
set 56,4 to 253,152 63241.0
set 56,5 to 252,151 63492.0
set 56,6 to 256,151 62500.0
set 56,7 to 252,152 63492.0
set 56,8 to 250,153 64000.0
set 56,9 to 252,151 63492.0
set 56,10 to 254,149 62992.0
set 56,11 to 253,152 63241.0
set 56,12 to 254,151 62992.0
set 56,13 to 252,152 63492.0
set 56,14 to 250,151 64000.0
set 56,15 to 252,154 63492.0
set 56,16 to 252,152 63492.0
set 56,17 to 254,150 62992.0
set 56,18 to 253,152 63241.0
set 56,19 to 250,151 64

In [5]:
testRuns(cobras)
len(cobras)

1

In [6]:
func.RUN(cobras)

False

In [6]:
%pdb on
func.run(cobras, [5], [5], [0.07], [0.03], dirs=('cw', 'cw'))
%pdb off

Automatic pdb calling has been turned ON


TypeError: unsupported operand type(s) for >>: 'float' and 'int'

> /Users/cloomis/Sumire/PFS/git/pfs_gui_revF/func.py(105)toList()
    103         c = 0x0000 | (self.dir[0]=='ccw') | ((self.dir[1]=='ccw')<<1) | \
    104                 (self.en[0]<<2) | (self.en[1]<<3) | (board<<4) | (cnum%30<<11)
--> 105         p = [c>>8, c%256, self.pulses[0]>>8, self.pulses[0]%256, self.steps[0]>>8, \
    106                 self.steps[0]%256, self.sleeps[0]>>8, self.sleeps[0]%256, \
    107                 self.pulses[1]>>8, self.pulses[1]%256, self.steps[1]>>8, \

ipdb> c


In [7]:
func.RUN(cobras, timeout=5)

TypeError: unsupported operand type(s) for >>: 'float' and 'int'

> /Volumes/ssdata/git/pfs_gui_revF/func.py(105)toList()
    103         c = 0x0000 | (self.dir[0]=='ccw') | ((self.dir[1]=='ccw')<<1) | \
    104                 (self.en[0]<<2) | (self.en[1]<<3) | (board<<4) | (cnum%30<<11)
--> 105         p = [c>>8, c%256, self.pulses[0]>>8, self.pulses[0]%256, self.steps[0]>>8, \
    106                 self.steps[0]%256, self.sleeps[0]>>8, self.sleeps[0]%256, \
    107                 self.pulses[1]>>8, self.pulses[1]%256, self.steps[1]>>8, \

ipdb> p c
2076
ipdb> p self.pulses
(0.01, 0.03)
ipdb> p self.steps
(1, 3)
ipdb> c


In [7]:
for i in range(10):
    func.RST()

In [26]:
func.EXIT()

True

In [10]:
%pdb on
setFreq(cobras)
%pdb off

Automatic pdb calling has been turned ON
setting 1, 63241.0, 104575.0


TypeError: unsupported operand type(s) for >>: 'numpy.float64' and 'int'

> /Users/cloomis/Sumire/PFS/git/pfs_gui_revF/func.py(124)toList()
    122         c = 0x0000 | (self.en[0]) | (self.en[1]<<1) | \
    123                 (board<<4) | (cnum%30<<11)
--> 124         p = [c>>8, c%256, self.m0Per>>8, self.m0Per%256, \
    125                 self.m1Per>>8, self.m1Per%256 ]
    126         return p

ipdb> c


In [19]:
print(calibs.motorFreq1, calibs.motorFreq2)
print(calibs.motorOntimeFwd1, calibs.motorOntimeFwd2)

[ 63241.  63241.  63745.  63492.  63241.  63492.  62500.  63492.  64000.
  63492.  62992.  63241.  62992.  63492.  64000.  63492.  63492.  62992.
  63241.  64000.  63745.  62992.  63492.  64000.  63241.  62500.  63241.
  63241.  63492.  63492.  63745.  63745.  63745.  63241.  63241.  64000.
  62745.  64000.  62745.  62992.  63492.  62500.  63241.  62256.  64000.
  63492.  64000.  62992.  63492.  64516.  64000.  64257.  64516.  63745.
  64000.  64777.  63492.] [ 108108.  104575.  103896.  104575.  105263.  105960.  105960.  105263.
  104575.  105960.  107382.  105263.  105960.  105263.  105960.  103896.
  105263.  106666.  105263.  105960.  105960.  104575.  104575.  105263.
  105960.  106666.  104575.  108108.  105960.  105263.  108843.  106666.
  105263.  105263.  106666.  105263.  105960.  105263.  104575.  106666.
  105960.  105263.  105960.  104575.  105960.  105263.  105960.  105263.
  106666.  106666.  105960.  105960.  104575.  104575.  103896.  107382.
  103896.]
[ 0.04572   0.

In [20]:
print(calibs.S1Nm[0])

[  70.03209804   62.74728531   56.84150693   57.90853668   57.06857741
   57.06857741   58.05328001   58.05328001   61.63222681   61.63222681
   61.63222681   69.17223887   76.50784205   76.50784205   77.79746726
  108.14058276  109.15375519  122.17056368   81.28062135   46.92816081
  118.288756    105.69583089   82.41003571  118.288756     73.87191431
  107.69414862  113.66147823  115.45492447  115.45492447  133.71466776
   97.19222462   80.79719903   85.51678267   95.06456468  167.15419975
  130.84248019  187.44142455  139.67022308   78.51347815   67.68316757
   67.50421901   49.18906363   51.51762332   81.73644537   73.56246679
   67.12785993   61.17559094   59.56714541   59.99700015   56.73401204
   59.70347275   57.34218952   64.28112278   69.29606745   52.83395462
   49.94381321   50.07720235   52.8262018    55.78022591   95.67852017
   74.02228893   68.79024707   68.02078413   72.07207207   70.29876977
   70.06480995   73.40646793   69.82698425   66.74577277   66.85857554
   44.